In [3]:
import requests
from bs4 import BeautifulSoup

# Gửi yêu cầu GET đến trang web
url = "https://www.presight.io/privacy-policy.html"  
response = requests.get(url)  # Lấy nội dung HTML của trang

soup = BeautifulSoup(response.content, 'html.parser')  # Sử dụng BeautifulSoup để phân tích HTML

# Trích xuất tất cả các thẻ <p> (đoạn văn)
paragraphs = soup.find_all('p')  
data = [p.get_text(strip=True) for p in paragraphs]  # Lấy nội dung văn bản từ các thẻ <p>, loại bỏ khoảng trắng

# In ra dữ liệu đã trích xuất
print("\n".join(data)) 


By Role
By Team
At Presight, we are committed to protecting the privacy of our customers and visitors to our website. This Privacy Policy explains how we collect, use, and disclose information about our customers and visitors.
We collect several different types of information for various purposes to provide and improve our Service to you.
While using our Service, we may ask you to provide us with certain personally identifiable information that can be used to contact or identify you ("Personal Data"). Personally identifiable information may include, but is not limited to:
We may also collect information that your browser sends whenever you visit our Service or when you access the Service by or through a mobile device ("Usage Data"). This Usage Data may include information such as your computer's Internet Protocol address (e.g. IP address), browser type, browser version, the pages of our Service that you visit, the time and date of your visit, the time spent on those pages, unique devic